In [4]:
import numpy as np
from sklearn import linear_model,datasets,tree
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.feature_selection import RFE

import sklearn.metrics as metrics
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from ggplot import *
from mpl_toolkits.mplot3d import axes3d, Axes3D 
from sklearn.tree import DecisionTreeClassifier
from pandas.tools.plotting import scatter_matrix
from sklearn.svm import SVC  
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA, KernelPCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
pd.set_option('display.max_columns', None)
from sklearn import preprocessing
from scipy.stats.stats import pearsonr 
from imblearn.under_sampling import RandomUnderSampler
import FCBF
import CFS
import reliefF
from id3 import Id3Estimator
from scipy import optimize
%matplotlib inline

c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\ggplot\utils.py:81: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access Timestamp as pandas.Timestamp
  pd.tslib.Timestamp,


In [6]:
iri = pd.read_excel('1_YEAR_2.FOLD.xlsx')

In [7]:
iris=iri #.select_dtypes(['number'])
for column in iris.columns:
    if iris[column].dtype == type(object):
        le = preprocessing.LabelEncoder()
        iris[column] = le.fit_transform(iris[column])
iri2=iris
#X=iri2.loc[:,['LIFE_SUP_TCR','HGT_CM_TCR','DAYS_STAT1','GENDER_MATCH','DEATH_MECH_DON','AGE','HLAMAT','AGE_MATCH_LEVEL','PROC_TY_HR','EDUCATION','SHARE_TY','GENDER','IMPL_DEFIBRIL','PULM_INF_DON','TATTOOS','ETHCAT_MATCH','AGE_DON','ABO','GENDER_DON','HIST_CIG_DON','DAYS_STAT1A','LIFE_SUP_TRR','MED_COND_TRR','CORONARY_ANGIO','VENT_SUPPORT_AFTER_LIST','ETHCAT_DON','HIST_OTH_DRUG_DON','DIAB','ABO_MAT','INIT_STAT','DDAVP_DON','END_STAT','DEATH_CIRCUM_DON','PRI_PAYMENT_TCR','ETHCAT','PRI_PAYMENT_TRR','REGION','ABO_DON','DIAG','THORACIC_DGN','FUNC_STAT_TCR','TCR_DGN','FUNC_STAT_TRR']]
#Uncoment for features as in spss
cols=["ABO","ABO_DON","ABO_MAT","AGE","AGE_DON","AGE_MATCH_LEVEL","CORONARY_ANGIO","DAYS_STAT1","DAYS_STAT1A","DAYS_STAT2","DDAVP_DON","DEATH_CIRCUM_DON","DEATH_MECH_DON","DIAB","DIAG","DON_RETYP","EDUCATION","END_STAT","ETHCAT","ETHCAT_MATCH","FUNC_STAT_TCR","FUNC_STAT_TRR","GENDER","GENDER_DON","GENDER_MATCH","HEP_C_ANTI_DON","HIST_CIG_DON","HIST_OTH_DRUG_DON","HLAMAT","IMPL_DEFIBRIL","INIT_STAT","INOTROPES_TCR","LIFE_SUP_TCR","MED_COND_TRR","PRI_PAYMENT_TCR","PRI_PAYMENT_TRR","PROC_TY_HR","REGION","SHARE_TY","TCR_DGN","THORACIC_DGN","VENT_SUPPORT_AFTER_LIST"]
#X=iri2.loc[:,cols]
#uncoment for all features - gtime
iri3=iri2.drop(labels=['GSTATUS','GTIME'], axis=1)
X=iri3
Y = iri2.GSTATUS.values
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
#X= SelectKBest(chi2, k=1).fit_transform(X, Y)
#import seaborn as sns
corr = X.corr()
#corr= pearsonr(X,X)
col=list(X.columns.values)
col2=col #[:-2]
#fig, ax = plt.subplots()
# the size of A4 paper
#fig.set_size_inches(40, 30)
#sns.heatmap(corr, 
#        xticklabels=col2,
#        yticklabels=col2,ax=ax)
#fig.savefig('example.png')
unique, counts = np.unique(Y, return_counts=True)
target_count=dict(zip(unique, counts))
print('Class 0:', target_count[0])
print('Class 1:', target_count[1])
print('Proportion:', target_count[0] / target_count[1], ': 1')
#dictionary = plt.figure()
#plt.bar(range(len(target_count)), target_count.values(), align='center')
#plt.xticks(range(len(target_count)), target_count.keys())
#plt.show()
#X=X.values


Class 0: 10893
Class 1: 1455
Proportion: 7.48659793814433 : 1


In [4]:
#pd.plotting.scatter_matrix(X)
#plt.show()

In [5]:
def evaluate_threshold(threshold,tpr,fpr,thresholds):
    seng=tpr[thresholds > threshold][-1]
    speg=1 - fpr[thresholds > threshold][-1]
    print('Sensitivity:', seng)
    print('Specificity:', speg)
    return seng,speg
def acuracy_auc_and_confusionmatrix(model,x_test,y_test,graph):
    test_set_predictions = [model.predict(x_test[i].reshape((1,x_test.shape[1]))) for i in range(x_test.shape[0])]
    acg=accuracy_score(y_test, test_set_predictions)
    print("accuracy is : ",acg)
    print(confusion_matrix(y_test, test_set_predictions))
    fpr, tpr, thresholds = metrics.roc_curve(y_test, test_set_predictions)
    if graph==1:
        plt.plot(fpr, tpr)
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.0])
        plt.title('ROC curve for diabetes classifier')
        plt.xlabel('False Positive Rate (1 - Specificity)')
        plt.ylabel('True Positive Rate (Sensitivity)')
        plt.grid(True)
    seng,speg=evaluate_threshold(0.5,tpr,fpr,thresholds)
    aucg=metrics.roc_auc_score(y_test, test_set_predictions)
    print(aucg)
    return acg,aucg,seng,speg

In [6]:
def nocross(X,Y):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=10)
    return X_train, X_test, Y_train, Y_test
def smotefun(X_train,Y_train):
    sm = SMOTE(random_state=2)
    X_train, Y_train = sm.fit_sample(X_train, Y_train)
    return X_train, Y_train
def rus(X_train,Y_train):
    sm = RandomUnderSampler(random_state=2,)
    X_train, Y_train = sm.fit_sample(X_train, Y_train)
    return X_train, Y_train
def normalize(X_train,X_test):
    X_train = preprocessing.normalize(X_train, norm='l2')
    X_test = preprocessing.normalize(X_test, norm='l2')
    return X_train, X_test
def standardise(X_train,X_test):
    X_train = preprocessing.scale(X_train)
    X_test = preprocessing.scale(X_test)
    return X_train, X_test
def fitpredict(X_train,Y_train,mod,X_test,Y_test,graph):
    mod.fit(X_train,Y_train) 
    #export_graphviz(mod, 'tree3.dot', col)
    #rfe = RFE(estimator=mod, n_features_to_select=5, step=1)
    #rfe = rfe.fit(X_train, Y_train)
    #print('Chosen best 5 feature by rfe:',rfe.support_)
    acg,aucg,seng,speg=acuracy_auc_and_confusionmatrix(mod,X_test,Y_test,graph)
    return acg,aucg,seng,speg
def modelselector(model):
    if model==0:
        print("LR")
        mod = linear_model.LogisticRegression(C=1e5) #C is the inverse of the regularization factor
    if model==1:
        print("ANN")
        mod = MLPClassifier(hidden_layer_sizes=(30,30,30),max_iter=200000)
    if model==2:
        print("CRT")
        mod = DecisionTreeClassifier(criterion = "gini")
    if model==3:
        print("SVCL")
        mod = SVC(kernel='linear')  
    if model==4:
        print("SVCP")
        mod = SVC(kernel='poly', degree=2)    
    if model==5:
        print("Random Forest")
        mod = RandomForestClassifier(random_state=43)
    if model==6:
        print("DT")
        mod = Id3Estimator()
    return mod
def stdselector(X_train,X_test,norstd):
    if norstd==1:
        print("Normalize")
        X_train,X_test=normalize(X_train,X_test)
    if norstd==2:
        print("Standardize")
        X_train,X_test=standardise(X_train,X_test)    
    return X_train,X_test
def smoteselector(X_train,X_test,Y_train,Y_test,smote):
    if smote==1:
        print("Smote")
        X_train, Y_train=smotefun(X_train,Y_train)
    if smote==2:
        print("RUS")
        X_train, Y_train=rus(X_train,Y_train)
    return X_train,X_test,Y_train,Y_test

def crosse(X,Y,norstd,smote,featureselect,model,graph) :
    X=X.values
    ac=[]
    auc=[]
    sen=[]
    spe=[]
    kf = model_selection.StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    for train_index, val_index in kf.split(X, Y):
        X_train, X_test = X[train_index], X[val_index]
        Y_train, Y_test = Y[train_index], Y[val_index]
        X_train,X_test=stdselector(X_train,X_test,norstd)
        X_train,X_test,Y_train,Y_test=smoteselector(X_train,X_test,Y_train,Y_test,smote)
        X_train,X_test=featureselector(featureselect,X_train,Y_train,X_test)
        mod=modelselector(model)
        acg,aucg,seng,speg=fitpredict(X_train,Y_train,mod,X_test,Y_test,graph)
        ac.append(acg)
        auc.append(aucg)
        sen.append(seng)
        spe.append(speg)
    print("10foldcrossvalidation mean AUC",np.mean(auc))
    print("10foldcrossvalidation mean ACCURACY",np.mean(ac))
    print("10foldcrossvalidation mean SENSITIVITY",np.mean(sen))
    print("10foldcrossvalidation mean SPECIFICITY",np.mean(spe))
def fcb(X_train,Y_train,X_test):
    #X_train=X_train.values
    #X_test=X_test.values
    idx =FCBF.fcbf(X_train,Y_train)
    for i in idx[0]:
        print('Using ',col[i])
    features = X_train[:, idx[0]]
    features2 = X_test[:, idx[0]]
    return features,features2
def cf(X_train,Y_train,X_test):
    #X_train=X_train.values
    #X_test=X_test.values
    idx =CFS.cfs(X_train,Y_train)
    for i in idx:
        print('Using ',col[i])
    features = X_train[:, idx]
    features2 = X_test[:, idx]
    return features,features2
def rel(X_train,Y_train,X_test):
    score = reliefF.reliefF(X_train,Y_train)
    idx = reliefF.feature_ranking(score)
    for i in idx:
        print('Using ',col[i])
    features = X_train[:, idx]
    features2 = X_test[:, idx]
    return features,features2
def sa(X_train,Y_train,X_test):
    #X_train=X_train.values
    #X_test=X_test.values
    pca = PCA(n_components=5)
    X_train = pca.fit_transform(X_train)
    X_test = pca.fit_transform(X_test)
    pca_result=X_train
    df=pd.DataFrame(pca_result)
    scatter_matrix(df, alpha=0.2, figsize=(6, 6), diagonal='kde')
    print(X_test.shape)
    features = X_train
    features2 = X_test
    return features,features2    
def pc(X_train,Y_train,X_test):
    #X_train=X_train.values
    #X_test=X_test.values
    pca = PCA(n_components=5)
    X_train = pca.fit_transform(X_train)
    X_test = pca.fit_transform(X_test)
    pca_result=X_train
    df=pd.DataFrame(pca_result)
    scatter_matrix(df, alpha=0.2, figsize=(6, 6), diagonal='kde')
    print(X_test.shape)
    features = X_train
    features2 = X_test
    return features,features2    
def featureselector(featureselect,X_train,Y_train,X_test):
    if featureselect==1:
        print("FCBF")
        X_train,X_test=fcb(X_train,Y_train,X_test)
    if featureselect==2:
        print("CFS")
        X_train,X_test=cf(X_train,Y_train,X_test)
    if featureselect==3:
        print("ReliefF")
        X_train,X_test=rel(X_train,Y_train,X_test)
    if featureselect==4:
        print("PCA")
        X_train,X_test=pc(X_train,Y_train,X_test)
    if featureselect==5:
        print("Simul")
        X_train,X_test=sa(X_train,Y_train,X_test)
    return X_train,X_test
def mod(cross,norstd,smote,featureselect,model,graph):
    if cross==0:
        X_train, X_test, Y_train, Y_test=nocross(X,Y)
        X_train,X_test=stdselector(X_train,X_test,norstd)
        X_train,X_test,Y_train,Y_test=smoteselector(X_train,X_test,Y_train,Y_test,smote)
        X_train,X_test=featureselector(featureselect,X_train,Y_train,X_test)
        mod=modelselector(model)
        fitpredict(X_train,Y_train,mod,X_test,Y_test,graph)
    if cross==1:
        print("10 Cross")
        crosse(X,Y,norstd,smote,featureselect,model,graph)
def pca(comp):
    a=PCA(n_components=comp)
    return a
def kpca(comp,kernel_name,gamma):
    a=KernelPCA(n_components=comp,kernel=kernel_name,gamma=gamma)
    return a
def lda(comp):
    a=LinearDiscriminantAnalysis(n_components=comp, priors=None, shrinkage=None,solver='svd', store_covariance=False, tol=0.0001)
    return a
def dim(dimen,comp,kernel_name,gamma):
    if dimen==0:
        a=pca(comp)
    if dimen==1:
        a=kpca(comp,kernel_name,gamma)
    if dimen==2:
        a=lda(comp)
    return a

def pca(dimen,kernel_name,gamma,comp,smote,model):
    comp=int(comp)
    Xs=preprocessing.scale(X)
    kpca=dim(dimen,comp,kernel_name,gamma)
    #kpca = KernelPCA(n_components=comp,kernel=kernel_name,gamma=gamma)
    if dimen==2:
        X_transformed = kpca.fit_transform(X)
    else:
        X_transformed = kpca.fit_transform(X)
    X_train, X_test, Y_train, Y_test=nocross(X_transformed,Y)
    X_train,X_test,Y_train,Y_test=smoteselector(X_train,X_test,Y_train,Y_test,smote)
    mod=modelselector(model)
    fitpredict(X_train,Y_train,mod,X_test,Y_test,1)



In [7]:
#for i in range(0,6):
#mod(cross,norstd,smote,featureselect,model,graph)
#cross:0 for normal // 1 for 10cross validation
#norstd:0 for no effect // 1 for normalize // 2 for standardize
#smote:0 for no effect // 1 for smote // 2 for rus
#featureselect:0 for no effect // 1 for fcbf // 2 for cfs (takes long time)// 3 for reliefF (takes long time)// 4 for PCA
#model:0 for no LR // 1 for ANN // 2 for CRT // 3 for SVCL(SVM) // 4 for SVCP(SVM POLY) // 5 for Random Forest // 6 for DT
#run nested loop to print all
#mod(0,0,0,5,1,0)
#pca(X,Y)
from ipywidgets import interact, FloatSlider, Dropdown,IntSlider
interact(pca, 
         dimen=IntSlider(min=0, max=2, step=1, value=1),kernel_name=Dropdown(options=['rbf', 'sigmoid']),gamma=FloatSlider(min=0.001, max=25, step=0.001, value=10),C=FloatSlider(min=0.001, max=10, step=0.001, value=1),comp=FloatSlider(min=1, max=10, step=1, value=5),smote=IntSlider(min=0, max=2, step=1, value=1),model=IntSlider(min=0, max=10, step=1, value=2))

interactive(children=(IntSlider(value=1, description='dimen', max=2), Dropdown(description='kernel_name', opti…

<function __main__.pca(dimen, kernel_name, gamma, comp, smote, model)>

In [31]:
def my_input_fn(features, targets, batch_size=1, shuffle=True, num_epochs=None):
    # Convert pandas data into a dict of np arrays.
    features = {key:np.array(value) for key,value in dict(features).items()}                                            
 
    # Construct a dataset, and configure batching/repeating.
    ds = Dataset.from_tensor_slices((features,targets)) # warning: 2GB limit
    ds = ds.batch(batch_size).repeat(num_epochs)
    
    # Shuffle the data, if specified.
    if shuffle:
        ds = ds.shuffle(10000)
    
    # Return the next batch of data.
    features, labels = ds.make_one_shot_iterator().get_next()
    return features, labels
def train_linear_classifier_model(
    learning_rate,
    steps,
    batch_size,
    training_examples,
    training_targets,
    validation_examples,
    validation_targets):
    periods = 10
    steps_per_period = steps / periods
    print(steps_per_period)
  
  # Create a linear classifier object.
    my_optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)  
    linear_classifier = tf.estimator.LinearClassifier(feature_columns=training_examples,optimizer=my_optimizer)
  
  # Create input functions.
    training_input_fn = lambda: my_input_fn(training_examples,training_targets, batch_size=batch_size)
    predict_training_input_fn = lambda: my_input_fn(training_examples,training_targets,num_epochs=1,shuffle=False)
    predict_validation_input_fn = lambda: my_input_fn(validation_examples, validation_targets, num_epochs=1, shuffle=False)
  
  # Train the model, but do so inside a loop so that we can periodically assess
  # loss metrics.
    x,y=training_input_fn()
    print(x,y)
    print("Training model...")
    print("LogLoss (on training data):")
    training_log_losses = []
    validation_log_losses = []
    for period in range (0, periods):
    # Train the model, starting from the prior state.
        linear_classifier.train(input_fn=training_input_fn,steps=steps_per_period)
    # Take a break and compute predictions.    
        training_probabilities = linear_classifier.predict(input_fn=predict_training_input_fn)
        training_probabilities = np.array([item['probabilities'] for item in training_probabilities])
    
        validation_probabilities = linear_classifier.predict(input_fn=predict_validation_input_fn)
        validation_probabilities = np.array([item['probabilities'] for item in validation_probabilities])
    
        training_log_loss = metrics.log_loss(training_targets, training_probabilities)
        validation_log_loss = metrics.log_loss(validation_targets, validation_probabilities)
    # Occasionally print the current loss.
        print("  period %02d : %0.2f" % (period, training_log_loss))
    # Add the loss metrics from this period to our list.
        training_log_losses.append(training_log_loss)
        validation_log_losses.append(validation_log_loss)
    print("Model training finished.")
  
  # Output a graph of loss metrics over periods.
    plt.ylabel("LogLoss")
    plt.xlabel("Periods")
    plt.title("LogLoss vs. Periods")
    plt.tight_layout()
    plt.plot(training_log_losses, label="training")
    plt.plot(validation_log_losses, label="validation")
    plt.legend()

    return linear_classifier

In [32]:
from __future__ import print_function

import math

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=10)
linear_classifier = train_linear_classifier_model(
    learning_rate=0.000005,
    steps=500,
    batch_size=20,
    training_examples=X_train,
    training_targets=Y_train,
    validation_examples=X_test,
    validation_targets=Y_test)

50.0
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\ADMIN\\AppData\\Local\\Temp\\tmpxn0wjma5', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000195EEC45F98>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
{'ABO': <tf.Tensor 'IteratorGetNext:0' shape=(?,) dtype=int32>, 'ABO_DON': <tf.Tensor 'IteratorGe

INFO:tensorflow:Calling model_fn.


ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().